# Tutorial de 5 minutos

La manera mas accesible de usar `pydap` es como `cliente` de acceso a datos cientificos en servidores remotos de [OPeNDAP](https://www.opendap.org/). 

## OPeNDAP - la visión original
La vision original de [OPeNDAP](https://www.opendap.org/) ([Cornillion, et al 1993](https://zenodo.org/records/10610992)) fue el hacer las equivalencias

$ \;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\; \boxed{\text{URL} \approx \text{Dataset Remoto} }$

$ \;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\;\; \boxed{\text{URL + Expresión de Restricción} \approx \text{Subregion de un Dataset Remoto}} $

Esa vision original fue la que conllevo el desarrollo del protocolo `DAP2`. En la actualidad, tanto <span style='color:#ff6666'>**OPeNDAP**</span> como Unidata implementan el protocolo `DAP4`, el cual es mas moderno y abarca mas typos de information, y cubre todos los elementos esenciales cubiertos por el protocolo `DAP2` (para mas informacion vea [DAP4 specification](https://opendap.github.io/dap4-specification/DAP4.html#_how_dap4_differs_from_dap2)).

## La aportación de PyDAP:

The logica interna de `PyDAP` permite  la construccion de expressiones de restriccion (`CE`s for su siglas en ingles) para cada url, de una manera interactiva, facilitando el accesso a subconjunto de datos remotos atraves de <span style='color:#ff6666'>**OPeNDAP**</span>. Ademas, como `pydap` es un "[backend engine](https://docs.xarray.dev/en/stable/user-guide/io.html#opendap)" del paquete de Python `Xarray`, usuarios pueden escalar su flujo de trabajo con la combinacion `Xarray`+`PyDAP` y el uso de parallelismo que `Xarray` permite. En general, un dominio basico del uso de expressiones de restriction (`CE`s) es importante para maximizar los protocolos de OPeNDAP.


### Objetivos:


- Demonstrar como especificar el protocolo <span style='color:#0066cc'>**DAP4**</span> al servidor remoto de <span style='color:#ff6666'>**OPeNDAP**</span>.
- El uso de `Xarray` y `PyDAP` para descargar un subcojunto de datos remotos en 2 escenario typicos: `a)` Un archivo remoto con extension `NcML` que representa un archivo virtua l de aggregacion , y `b)` dos archivos remotos de format Netcdf.
- Demonstrat las distintas maneras en que pueden user las Condiciones de Restriccion (`CE`s), y como estas se pueden pasar al servidor para que cualquier operacion de extraer subconjuntos <span style='color:#0066cc'>**sea hecha por el servidor OPeNDAP**</span>, de una manera `proxima` a los archivos remotos, de una manera eficiente.


### Requiremientos

- Archivos expuestos por un servidor OPeNDAP que implemente el protocolo <span style='color:#0066cc'>**DAP4**</span>. Por ejemplo, el servidor: http://test.opendap.org/opendap/. 
- pydap>=3.5.8
- xarray>=2025.0
- numpy>=2.0

```{note}
The gran mayoria de los servidores OPeNDAP de la NASA implementan el protocolo <span style='color:#0066cc'>**DAP4**</span>.
```



In [ ]:
from pydap.client import open_url, consolidate_metadata, create_session
import xarray as xr
import numpy as np

In [ ]:
# create a session to inspect downloads. cache_name must have `debug`
session = create_session(use_cache=True, cache_kwargs={"cache_name":'data/debug_case1'})
session.cache.clear()

## 1) Accesso a subconjuntos de un archivo NcML file

El archivo que utilizaremos tiene formate `NcML` representando una aggregacion virtual de muchos archivos dataset, el cual puede ser encontrado en el servidos prueba con nombre: [aggExisting.ncml](http://test.opendap.org/opendap/data/ncml/agg/aggExisting.ncml.dmr.html).

Los servidores de <span style='color:#ff6666'>**OPeNDAP**</span> pueden ser configurados para producir estos archivos de aggregation con formato `NcML`. La ventaja es que el usuario trabaja con un solo URL para toda la informacion posible, mientras que en el escenario cuando los archivos no has sido aggregados, el usuario tiene entonces que trabajar con multiples (a veces miles) de URLs.


In [ ]:
ncml_url = "http://test.opendap.org/opendap/data/ncml/agg/aggExisting.ncml"
dap4_ncml_url = ncml_url.replace("http",  "dap4")
print("=============================================================\n URL DAP4: \n", dap4_ncml_url, "\n=============================================================")

Ahora utilizamos Xarray y Pydap para "abrir" el archivo, como si estuviera en tu computadora de trabajo. Para esto ejecte el siguiente bloque de codigo

In [ ]:
ds = xr.open_dataset(
    dap4_ncml_url, 
    engine='pydap',
    session = session,
    chunks={},
)
ds

```{note}
El archivo sigue estando remoto, pero el programa de Xarray junto con Pydap permiten accesar a la information dentro archivo, como si estuviera accesando un archivo dentro de su computadora.
```

### Como dercargamos un elemento de una variable del archivo remoto?

Para demostrar como descargar datos, primero inspeccionamos la descripcion de la variable `T`. Es importante, antes que nada, entender la estructura interna del archivo.


In [ ]:
ds['T']

```{note}
La informacion que describe la variable `T` implica que toda `T` esta contenida en un solo `chunk`. `Xarray` y `OPeNDAP` en general transmiten y realizan operaciones dividiento la informacion del archivo en `chunks`. Y `Xarray` interpreta a toda variable dentro de un archivo remoto en OPeNDAP, como un solo `chunk`, incluso cuando el archivo remoto divide la representacion de cada variable en diversos chunks. 
```


In [ ]:
# clear the cache to inspect what is being downloaded
session.cache.clear() 

In [ ]:
ds['T'].isel(time=1, lon=0, lat=0).load()

In [ ]:
print("====================================== \n Solicitud enviada al Servidor OPeNDAP \n ", session.cache.urls()[0].split("?")[-1].split("&dap4.checksum")[0].replace("%5B","[").replace("%5D","]").replace("%3A",":").replace("%2F","/"), "\n====================================== ")

<span style='color:#0066cc'>**La expression de restriccion (CE) fue contruida por el metodo**<span style='color:black'>
`.isel` de `Xarray`. Este <span style='color:#0066cc'>**metodo interno de Xarray fue entonces enviada al servidor OPeNDAP, el cual hizo todo el trabajo por nosotros!**<span style='color:black'>

## 2) Accesando a subconjuntos en 2 archivos remotos que pertenecen al mismo proyecto

En este escenario, los dos archivos remotos describen informacion contigua del mismo proyecto, bajo la suposicion que estos dos archivos pueden aggregarse a lo largo de una dimension. Por ejemplo, un archivo representa los valores de variables geofisicas en la fecha 10/Sept/2025, y el segundo archivo representa valores de las mismas variables geofisicas en la siguiente fecha disponible.

Utilizaremotes para este ejemplo los siguientes archivos: [coads_climatology](http://test.opendap.org/opendap/data/nc/coads_climatology.nc.dmr.html) and [coads_climatology2](http://test.opendap.org/opendap/data/nc/coads_climatology.nc.dmr.html). Estos dos archivos abarcan la misma cobertura espacial, y pueden ser aggregados en tiempo.

```{note}
Es importante verificar siempre que los conjuntos de datos se puedan agregar. `PyDAP` y `Xarray` contienen lógica interna que verifica si dos o más conjuntos de datos se pueden concatenar. Sin embargo, estas comprobaciones de seguridad solo consideran dimensiones y coordenadas.
```

<span style='color:#0066cc'>**Un paso importante será el uso de Expresiones de Restricción (CE) para garantizar que solo se concatenen las variables de interés**<span style='color:black'>.

```{warning}
Uno de estos archivos tiene variables adicionales que no están presentes en el otro archivo y que descartaremos mediante el uso de CE.
```


In [ ]:
urls = ["http://test.opendap.org/opendap/data/nc/coads_climatology.nc", "http://test.opendap.org/opendap/data/nc/coads_climatology2.nc"]
dap4_urls = [url.replace("http","dap4") for url in urls]

# Expression de Restriccion (CE)
dap4_CE = "?dap4.ce=" + ";".join(["/SST", "/COADSX", "/COADSY", "/TIME"])

# Final list of OPeNDAP URLs
dap4ce_urls =[url+dap4_CE for url in dap4_urls]
print("====================================================\nURLs de OPeNDAP con protocolo DAP4 \n", dap4ce_urls)



```{note}
**Q:¿Por qué usar `CE`s cuando `Xarray` tiene un método `.drop_variables`?** Porque `Xarray` necesita analizar primero todos los metadatos remotos para luego descartar las variables. En algunos archivos, es posible encontrar hasta 1000 variables. `Xarray` las analizaría todas y luego las descartaría. Con `CE`s, el servidor envía metadatos restringidos asociados únicamente a las variables deseadas. Asi, `Xarray` solo procesa las variables de importancia.
```


```{warning}
`Xarray` espera la presencia de dimensiones en los metadatos. Al construir la `CE`, el usuario debe asegurarse de incluir todas las dimensiones asociadas con las variables de interés. En el ejemplo anterior, `COASX`, `COADSY` y `TIME` son las dimensiones de `SST`.
```



### <span style='color:#0066cc'>**Consolidate Metadata acelera el proceso de abrir una serie de archivos**<span style='color:black'>.


In [ ]:
consolidate_metadata(dap4ce_urls, session=session, concat_dim="TIME")

```{note}
`consolidate_metadata(dap4_urls, concat_dim='...', session=session)` descarga las dimensiones del archivo remoto y las almacena en formato SQLite, para su reuso. Esto significa que el objecto `session` permite autentificar y actua como un database manager! ¡Esta práctica puede resultar en una mejora del rendimiento en flujos de trabajo entre 10 y 100 veces más rápidos!
```

### Usamos Xarray como herramienta para abrir, descargar, y almacenar la informacion remota.

Internamente, `Xarray` utiliza `pydap` para comunicar con el servidor de OPeNDAP.


In [ ]:
ds = xr.open_mfdataset(
    dap4ce_urls, 
    engine='pydap',
    concat_dim='TIME',
    session=session,
    combine="nested",
    parallel=True,
    decode_times=False,
)
ds

In [ ]:
ds['SST']

### <span style='color:#0066cc'>**Que pasa si queremos descargar un solo elemento**<span style='color:black'>


In [ ]:
session.cache.clear()

In [ ]:
%%time
ds['SST'].isel(TIME=0, COADSX=0, COADSY=0).load() # this should download a single point one of the files

In [ ]:
print("====================================== \n Solicitud enviada al Servidor OPeNDAP:\n ", session.cache.urls()[0].split("?")[-1].split("&dap4.checksum")[0].replace("%5B","[").replace("%5D","]").replace("%3A",":").replace("%2F","/"), "\n====================================== ")

### <span style='color:#0066cc'>**Toda la variable fue descargada innecessariamente<span style='color:black'>** !!

Lo que queremos, es ver que la solicitud enviada al servidor OPeNDAP contenga la siguiente CE:

```python
dap4.ce=/SST[0:1:0][0:1:0][0:1:0]
```

`xr.open_mfdataset` no pasa el argumento de seleccion al servidor, de la misma manera en que `xr.open_dataset` lo hace. En su lugar, `Xarray` solicita toda la variable, y ya descargada, `Xarray` hace la selection localmente, de acuerdo al argumento `.isel` proporcionado por el usuario.



### <span style='color:#0066cc'>**Como asegurar que la selection is enviada al servidor OPeNDAP?<span style='color:black'>**


**La respuesta es proporcional le argumento extra, `chunk`, cuando abrimos/creamos el dataset con Xarray**. Este argumento `chunk` **debe ser igual al tamano de selection que esperamos como resultado final**. 
```{warning}
Si el argumento `chunk` is mas pequeno que el tamano que esperanos de nuestra descarga, `Xarray` terminara enviando muchas solicitudes de descargas innecesarias al servidor, para luego juntar todos los subconjuntos descargados. Este flujo de trabajo tampoco es idea, pues `Xarray` termina haciendo trabajo extra. Lo ideal es hacer que el servidor haga todo el trabajo, y `Xarray` solo proporciona el parallelismo.
```

A continuacion demostramos lograr que Xarray pase la selection al servidor remoto OPeNDAP, el cual hace la mayoria del trabajo de selection cerca del archivo remoto, y solo envia la informacion requerida.



In [ ]:
# consolidate metadata again, since the cached metadata was cleared before
consolidate_metadata(dap4ce_urls, session=session, concat_dim="TIME")


In [ ]:
# For a single element in all dimensions, the expected size of the download is:
expected_sizes = {"TIME":1, "COADSX":1, "COADSY":1}

In [ ]:
%%time
ds = xr.open_mfdataset(
    dap4ce_urls, 
    engine='pydap',
    concat_dim='TIME',
    session=session,
    combine="nested",
    parallel=True,
    decode_times=False,
    chunks=expected_sizes, # <---------
)


In [ ]:
session.cache.clear()

In [ ]:
ds['SST'] # inspect chunks before download

In [ ]:
%%time
ds['SST'].isel(TIME=0, COADSX=0, COADSY=0).load() # triggers download of an individual chunk

In [ ]:
print("====================================== \n Solicitud enviada al Servidor OPeNDAP:\n ", session.cache.urls()[0].split("?")[-1].split("&dap4.checksum")[0].replace("%5B","[").replace("%5D","]").replace("%3A",":").replace("%2F","/"), "\n====================================== ")

### Warning: Be cautious about chunking

¡Ahora solo descargamos exactamente lo que solicitamos! Sin embargo, en algunos casos, el tiempo de descarga puede ser hasta diez veces más lento que cuando solicitamos más datos. La razón de esta lentitud se puede atribuir a la cantidad de fragmentos (`chunks`) que generó el gráfico de Dask.

Entonces, estos dos son los escenarios que experimentados al tratar de descargar un subconjunto de datos. 


* `No chunk definido. Xarray descarga toda la variable.`
* Chunk definido. Xarray descarga solo el elemento deseado del archivo remoto. Pero durante este processo,  `388800` chunks fueron creados!

Idealmente, el `Chunk manager` solo debería activar la descarga de un único fragmento. Sin embargo, se crearon 388800 para garantizar la transferencia del elemento al servidor. Esto, en ocasiones, puede provocar que el cliente (Xarray) tarde mas de lo normal.

En el escenario anterior, llegamos a extremos. Es mejor encontrar un punto medio para los chunks. Lo demostramos a continuación, pero ahora con subconjuntos a lo largo del tiempo.


In [ ]:
consolidate_metadata(dap4ce_urls, session=session, concat_dim="TIME")

In [ ]:
download_sizes = {"COADSY":1} # note that we will subset across all time

In [ ]:
%%time
ds = xr.open_mfdataset(
    dap4ce_urls, 
    engine='pydap',
    concat_dim='TIME',
    session=session,
    combine="nested",
    parallel=True,
    decode_times=False,
    chunks=download_sizes,
)
session.cache.clear()

In [ ]:
ds['SST']

In [ ]:
%%time
ds['SST'].isel(COADSX=0, COADSY=0).load()

In [ ]:
print("====================================== \n Solicitudes enviadas al Servidor OPeNDAP:\n ",  [url.split("?")[-1].split("&dap4.checksum")[0].replace("%5B","[").replace("%5D","]").replace("%3A",":").replace("%2F","/") for url in session.cache.urls()], "\n====================================== ")

### Exito! Similar tiempo descarga, pero descarga es mucho menor que antes!
